The  High Phi Motion illusion, is the illusory perception of a strong shift of motion induced by a slow inducing motion. A demo page is available on the [min author's webpage](http://lab-perception.org/demo/highphi/) and the effect is described in this excellent paper :
    Wexler M, Glennerster A, Cavanagh P, Ito H & Seno T (2013). Default perception of high-speed motion. PNAS, 110, 7080-7085. http://wexler.free.fr/papers/highphi.pdf


In this notebook, we will generate an extension of this illusion to answer to the question of knowing if it limited to the one-dimensional motion along the ring or if this can extended to arbitrary, 2D, planar motions.

In [1]:
from IPython.display import Video

prefix = '2025-06-09_extending-the-high-phi-illusion'
# Video(f'../files/{prefix}/high-phi.mp4', html_attributes="loop=True autoplay=True  controls=True")


<!-- TEASER_END -->

Let's first initialize the notebook:

In [2]:
%ls ../files/{prefix}/

high-phi-diagonal-inducer.mp4  high-phi-short.mp4
high-phi-diagonal.mp4          high-phi.mp4
high-phi-long.mp4              inducer.mp4
high-phi-short-inducer.mp4     shuffle.mp4
high-phi-short-shuffle.mp4


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt

## creating a textured motion




As a generic visual texture, let's synthetize a [Motion Clouds](https://neuralensemble.github.io/MotionClouds) for the inducer:

In [4]:
# %pip install MotionClouds

In [5]:
import MotionClouds as mc
mc.figpath = os.path.join('../files/', prefix)
os.makedirs(mc.figpath, exist_ok=True)

image_size_az, image_size_el, N_frame = 512, 512, 64
N_frame_inducer = 48
fx, fy, ft = mc.get_grids(image_size_az, image_size_el, N_frame)


In [6]:
# HACK
%rm -f ../files/{prefix}/*mp4 

In [7]:
name = 'inducer'

opts = dict(sf_0=0.015, B_theta=np.inf, B_sf=0.015)
env = mc.envelope_gabor(fx, fy, ft, **opts)

mc.figures(env, name, do_figs=False, figpath=mc.figpath, verbose=True)
mc.in_show_video(name, figpath=mc.figpath)

Before Rectification of the frames
Mean= 0.0 , std= 5.873840747692175e-05 , Min= -0.0002725409587618176 , Max= 0.0002620943760986014  Abs(Max)= 0.0002725409587618176
After Rectification of the frames
Mean= 0.5 , std= 0.10776069722469707 , Min= 0.0 , Max= 0.980834839081296
percentage pixels clipped= 0.0


""


This can be accessed as a `numpy` array:

In [ ]:
movie_inducer = mc.rectif(mc.random_cloud(env))[:, :, :N_frame_inducer]
print(f'movie_inducer shape = {movie_inducer.shape}')

On the first two axis, the spatial axis of pixels ($x$ and $y$), on the third the temporal axis $t$.



### shuffled movie

This corresponds in Fourier space to a white noise in time and can be parameterized by an infinite bandwidth on the temporal frequency axis:

In [ ]:
name = 'shuffle'
N_frame_shuffle = 4
fx, fy, ft = mc.get_grids(image_size_az, image_size_el, N_frame)
env = mc.envelope_gabor(fx, fy, ft, B_V=np.inf, **opts)

mc.figures(env, name, do_figs=False, figpath=mc.figpath)
mc.in_show_video(name, figpath=mc.figpath)

Similarly, we get a movie:

In [ ]:
movie_shuffle = mc.rectif(mc.random_cloud(env))[:, :, :N_frame_shuffle]
print(f'movie_shuffle shape = {movie_shuffle.shape}')

we can now use these arrays and concatenate them:

In [ ]:
name = 'high-phi'

movie_blank = np.ones_like(movie_shuffle) * 0.5
movie_inducer_east = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, V_X=1, V_Y=0, **opts)))
movie_inducer_west = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, V_X=-1, V_Y=0, **opts)))
movie_inducer_north = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, V_X=0, V_Y=1, **opts)))
movie_inducer_south = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, V_X=0, V_Y=-1, **opts)))

movie_highphi = np.concatenate((movie_inducer_east, movie_shuffle, movie_blank, movie_inducer_north, movie_shuffle, movie_blank, 
                                movie_inducer_west, movie_shuffle, movie_blank, movie_inducer_south, movie_shuffle, movie_blank), axis=-1)
mc.anim_save(movie_highphi, os.path.join(mc.figpath, name), figpath=mc.figpath, verbose=False)
mc.in_show_video(name, figpath=mc.figpath)


### wrapping up and make a movie


Now that we have all elements, let's wrap them up in a single function and export the result as a

In [ ]:
UPSCALE = 1
image_size_az, image_size_el, N_frame = 256*UPSCALE, 256*UPSCALE, 64

def make_shots(figname, 
               N_frame_inducer=32, N_frame_shuffle=4, 
               image_size_az=image_size_az, image_size_el=image_size_el, N_frame=N_frame, 
               sf_0=opts['sf_0'], B_theta=opts['B_theta'], B_theta_inducer=opts['B_theta'], theta=np.pi/4, B_sf=opts['B_sf'],
               fps = 12 # frames per second
    ):


    fx, fy, ft = mc.get_grids(image_size_az, image_size_el, N_frame)

    opts = dict(sf_0=sf_0, theta=theta, B_theta=B_theta, B_sf=B_sf)
    movie_shuffle = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, B_V=np.inf, **opts)))[:, :, :N_frame_shuffle]
    movie_blank = np.ones_like(movie_shuffle) * 0.5

    opts.update(B_theta=B_theta_inducer)
    movie_inducer_east = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, V_X=1, V_Y=0, **opts)))[:, :, :N_frame_inducer]
    movie_inducer_west = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, V_X=-1, V_Y=0, **opts)))[:, :, :N_frame_inducer]
    movie_inducer_north = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, V_X=0, V_Y=1, **opts)))[:, :, :N_frame_inducer]
    movie_inducer_south = mc.rectif(mc.random_cloud(mc.envelope_gabor(fx, fy, ft, V_X=0, V_Y=-1, **opts)))[:, :, :N_frame_inducer]

    movie_highphi = np.concatenate((movie_inducer_east, movie_shuffle, movie_blank, movie_inducer_north, movie_shuffle, movie_blank, 
                                    movie_inducer_west, movie_shuffle, movie_blank, movie_inducer_south, movie_shuffle, movie_blank), axis=-1)
    
    fname = os.path.join(mc.figpath, figname)
    mc.anim_save(movie_highphi, fname, figpath=mc.figpath, fps=fps, verbose=False)
    return fname + mc.vext # returns filename


This function allows us to test different configurations.

What if the inducer is short in time ?

In [ ]:
figname = 'high-phi-short-inducer'
fname = make_shots(figname, N_frame_shuffle=32, N_frame_inducer=8)
mc.in_show_video(figname, figpath=mc.figpath)

What if the inducer is long in time but the shuffle is long ?

In [ ]:
figname = 'high-phi-short-shuffle'
fname = make_shots(figname, N_frame_shuffle=8, N_frame_inducer=32)
mc.in_show_video(figname, figpath=mc.figpath)

What if both are short in time ?

In [ ]:
figname = 'high-phi-short'
fname = make_shots(figname, N_frame_shuffle=8, N_frame_inducer=8)
mc.in_show_video(figname, figpath=mc.figpath)

What if both are long in time ?

In [ ]:
figname = 'high-phi-long'
fname = make_shots(figname, N_frame_shuffle=32, N_frame_inducer=32)
mc.in_show_video(figname, figpath=mc.figpath)

What if both are long in time ?

In [ ]:
figname = 'high-phi-diagonal'
fname = make_shots(figname, B_theta=.1)
mc.in_show_video(figname, figpath=mc.figpath)


In [ ]:
figname = 'high-phi-diagonal-inducer'
fname = make_shots(figname, B_theta_inducer=.1)
mc.in_show_video(figname, figpath=mc.figpath)


In [ ]:
figname = 'high-phi-diagonal-both'
fname = make_shots(figname, B_theta=.1, B_theta_inducer=.1)
mc.in_show_video(figname, figpath=mc.figpath)

## some book keeping for the notebook

In [ ]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,matplotlib,imageio  -r -g -b